In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL.Image as Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow_hub as hub

In [15]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=IMAGE_SHAPE+(3,))
])

In [3]:
data_dir = tf.keras.utils.get_file('flower_photos', 
                                   origin="./datasets/flower_photos", 
                                   cache_dir='.',
                                   untar=False)
data_dir

'.\\datasets\\flower_photos'

In [4]:
import pathlib
data_dir=pathlib.Path(data_dir)
data_dir

WindowsPath('datasets/flower_photos')

In [5]:
flowers_image_dic={
    'roses':list(data_dir.glob('roses/*')),
    'daisy':list(data_dir.glob('daisy/*')),
    'dandelion':list(data_dir.glob('dandelion/*')),
    'sunflowers':list(data_dir.glob('sunflowers/*')),
    'tulips':list(data_dir.glob('tulips/*'))
}

In [6]:
flowers_label_dic={
    'roses':0,
    'tulips':4,
    'dandelion':2,
    'sunflowers':3,
    'daisy':1
}

In [7]:
x,y=[],[]

for flower_name, images in flowers_image_dic.items():
    for image in images:
        img=cv2.imread(str(image))
        rig=cv2.resize(img,(224,224))
        x.append(rig)
        y.append(flowers_label_dic[flower_name])

In [8]:
x=np.array(x)
y=np.array(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)
X_trainsc=X_train/255
X_testsc=X_test/255

In [16]:
X_trainsc[0].shape

(224, 224, 3)

In [9]:
pred=classifier.predict(np.array([x[0],x[1]]))
pred=np.argmax(pred,axis=1)
pred

array([795, 880], dtype=int64)

In [10]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
modelwotoplayer=hub.KerasLayer(feature_extractor_model,input_shape=(224,224,3))

In [11]:
num_of_flowers = 5

model = tf.keras.Sequential([
  modelwotoplayer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(X_trainsc, y_train, epochs=30)

Epoch 1/30
86/86 [==============================] - 57s 106ms/step - loss: 0.7804 - accuracy: 0.7133
Epoch 2/30
86/86 [==============================] - 7s 79ms/step - loss: 0.4082 - accuracy: 0.8579
Epoch 3/30
86/86 [==============================] - 7s 79ms/step - loss: 0.3178 - accuracy: 0.8975
Epoch 4/30
86/86 [==============================] - 7s 78ms/step - loss: 0.2655 - accuracy: 0.9204
Epoch 5/30
86/86 [==============================] - 7s 77ms/step - loss: 0.2311 - accuracy: 0.9310
Epoch 6/30
86/86 [==============================] - 7s 77ms/step - loss: 0.2006 - accuracy: 0.9379
Epoch 7/30
86/86 [==============================] - 7s 79ms/step - loss: 0.1788 - accuracy: 0.9539
Epoch 8/30
86/86 [==============================] - 7s 79ms/step - loss: 0.1557 - accuracy: 0.9633
Epoch 9/30
86/86 [==============================] - 7s 78ms/step - loss: 0.1412 - accuracy: 0.9658
Epoch 10/30
86/86 [==============================] - 7s 76ms/step - loss: 0.1313 - accuracy: 0.9680
Epoch 1

In [13]:
model.evaluate(X_testsc,y_test)

29/29 [==============================] - 53s 205ms/step - loss: 0.3903 - accuracy: 0.8682


[0.3903443515300751, 0.8681917190551758]